In [41]:
import numpy as np
import matplotlib.pyplot as plt

import bilby

from gwbench import basic_relations as bsr
from gwbench import network

import math
PI = np.pi

In [37]:
times = [0,1e1,1e2,1e3,1e4]

In [45]:
math.degrees(PI-0.33916285222) - 90

70.56740000017476

## Bilby vs GWbench (3G)

### Bilby

In [61]:
minimum_frequency = 5
reference_frequency = 5.
duration = 2856
sampling_frequency = 2048
approximant = "IMRPhenomD"

ce_40_snrs_bilby = []
ce_20_snrs_bilby = []
et1_snrs_bilby = []
et2_snrs_bilby = []
et3_snrs_bilby = []

for time in times:
    injection_parameters = dict(
                                chirp_mass          = 30.9,
                                mass_ratio          = 0.8,
                                chi_1               = 0.0,
                                chi_2               = 0.0,
                                luminosity_distance = 400.0,
                                ra                  = np.pi/3,
                                dec                 = np.pi/3,
                                theta_jn            = np.pi/3,
                                psi                 = np.pi/3,
                                phase               = 0,
                                geocent_time        = time
    )

    # generate signal
    waveform_generator = bilby.gw.WaveformGenerator(
                                                    duration                      = duration,
                                                    sampling_frequency            = sampling_frequency,
                                                    frequency_domain_source_model = bilby.gw.source.lal_binary_black_hole,
                                                    waveform_arguments            = dict(waveform_approximant=approximant, reference_frequency=reference_frequency),
                                                    parameter_conversion          = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    )
    # ifos = bilby.gw.detector.InterferometerList(["H1", "L1"])
    ifos = bilby.gw.detector.InterferometerList(["CE-40_C", "CE-20_S", "ET_ET1", "ET_ET2", "ET_ET3"])
    ifos.set_strain_data_from_power_spectral_densities(
                                                        sampling_frequency = sampling_frequency,
                                                        duration           = duration,
                                                        start_time         = injection_parameters["geocent_time"] - duration + 2,
    )
    ifos.inject_signal(waveform_generator=waveform_generator, parameters=injection_parameters)
    for ifo in ifos:
        ifo.minimum_frequency = minimum_frequency

    ce_40_snrs_bilby.append(ifos[0].meta_data['optimal_SNR'])
    ce_20_snrs_bilby.append(ifos[1].meta_data['optimal_SNR'])
    et1_snrs_bilby.append(ifos[2].meta_data['optimal_SNR'])
    et2_snrs_bilby.append(ifos[3].meta_data['optimal_SNR'])
    et3_snrs_bilby.append(ifos[4].meta_data['optimal_SNR'])

ce_40_snrs_bilby = np.array(ce_40_snrs_bilby)
ce_20_snrs_bilby = np.array(ce_20_snrs_bilby)
et1_snrs_bilby = np.array(et1_snrs_bilby)
et2_snrs_bilby = np.array(et2_snrs_bilby)
et3_snrs_bilby = np.array(et3_snrs_bilby)

15:51 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
15:51 bilby INFO    : Injected signal in CE-40_C:
15:51 bilby INFO    :   optimal SNR = 1271.43
15:51 bilby INFO    :   matched filter SNR = 1271.71-1.97j
15:51 bilby INFO    :   chirp_mass = 30.9
15:51 bilby INFO    :   mass_ratio = 0.8
15:51 bilby INFO    :   chi_1 = 0.0
15:51 bilby INFO    :   chi_2 = 0.0
15:51 bilby INFO    :   luminosity_distance = 400.0
15:51 bilby INFO    :   ra = 1.0471975511965976
15:51 bilby INFO    :   dec = 1.0471975511965976
15:51 bilby INFO    :   theta_jn = 1.0471975511965976
15:51 bilby INFO    :   psi = 1.0471975511965976
15:51 bilby INFO    :   phase = 0
15:51 bilby INFO    :   geocent_time = 0
15:51 bilby INFO    : Injected signal in CE-20_S:
15:51 bilby INFO    :   optimal SNR = 874.30
15:51 bilby INFO    

### GWbench

In [62]:
# Choose detector network
# network_spec = ['aLIGO_H','aLIGO_L']
network_spec = ['CE-40_C','CE-20_S', 'ET_ET1', 'ET_ET2', 'ET_ET3']

# Choose waveform.
wf_model_name = 'lal_bbh'
wf_other_var_dic = {'approximant':'IMRPhenomD'}

# Frequency range
freq_range = np.arange(20.,1024.,2**-4)

# psd_dict  = {'psd_file':'aLIGO_O4_high_asd.txt','is_asd':1}
# user_psds = {'aLIGO_H':psd_dict, 'aLIGO_L':psd_dict}

net = network.Network(network_spec)
net.set_wf_vars(wf_model_name=wf_model_name, wf_other_var_dic=wf_other_var_dic)

ce_40_snrs_gwbench = []
ce_20_snrs_gwbench = []
et1_snrs_gwbench = []
et2_snrs_gwbench = []
et3_snrs_gwbench = []

for time in times:
    inj_params = {
        'Mc':    30.9,
        'eta':   0.247,
        'chi1x': 0.0,
        'chi1y': 0.0,
        'chi1z': 0.0,
        'chi2x': 0.0,
        'chi2y': 0.0,
        'chi2z': 0.0,
        'DL':    400,
        'tc':    time,
        'phic':  0.,
        'iota':  np.pi/3,
        'ra':    np.pi/3,
        'dec':   np.pi/3,
        'psi':   np.pi/3,
        'gmst0': bilby.gw.utils.greenwich_mean_sidereal_time(time)
        }

    net.set_net_vars(
        f=freq_range, inj_params=inj_params,
        deriv_symbs_string='Mc eta',
        use_rot=0, #user_psds=user_psds
        )

    net.setup_ant_pat_lpf_psds() 
    net.calc_det_responses()
    net.calc_snrs()

    ce_40_snrs_gwbench.append(net.detectors[0].snr)
    ce_20_snrs_gwbench.append(net.detectors[1].snr)
    et1_snrs_gwbench.append(net.detectors[2].snr)
    et2_snrs_gwbench.append(net.detectors[3].snr)
    et3_snrs_gwbench.append(net.detectors[4].snr)

ce_40_snrs_gwbench = np.array(ce_40_snrs_gwbench)
ce_20_snrs_gwbench = np.array(ce_20_snrs_gwbench)
et1_snrs_gwbench = np.array(et1_snrs_gwbench)
et2_snrs_gwbench = np.array(et2_snrs_gwbench)
et3_snrs_gwbench = np.array(et3_snrs_gwbench)

2023-06-26 15:52:02,503 - Network - INFO : PSDs loaded.
2023-06-26 15:52:02,506 - Network - INFO : Antenna patterns and LPFs loaded.
2023-06-26 15:52:02,521 - Network - INFO : Detector responses calculated.
2023-06-26 15:52:02,526 - Network - INFO : SNRs calculated.
2023-06-26 15:52:02,574 - Network - INFO : PSDs loaded.
2023-06-26 15:52:02,578 - Network - INFO : Antenna patterns and LPFs loaded.
2023-06-26 15:52:02,594 - Network - INFO : Detector responses calculated.
2023-06-26 15:52:02,599 - Network - INFO : SNRs calculated.
2023-06-26 15:52:02,646 - Network - INFO : PSDs loaded.
2023-06-26 15:52:02,650 - Network - INFO : Antenna patterns and LPFs loaded.
2023-06-26 15:52:02,665 - Network - INFO : Detector responses calculated.
2023-06-26 15:52:02,670 - Network - INFO : SNRs calculated.
2023-06-26 15:52:02,717 - Network - INFO : PSDs loaded.
2023-06-26 15:52:02,721 - Network - INFO : Antenna patterns and LPFs loaded.
2023-06-26 15:52:02,738 - Network - INFO : Detector responses calc

### Comparison

In [63]:
print('CE-40_C Bilby: '    + str(np.abs(ce_40_snrs_bilby)))
print('CE-40_C GWBENCH: '    + str(np.abs(ce_40_snrs_gwbench)))
print(' ')
print('CE-20_S Bilby: ' + str(np.abs(ce_20_snrs_bilby)))
print('CE-20_S GWBENCH: ' + str(np.abs(ce_20_snrs_gwbench)))
print(' ')
print('ET_ET1 Bilby: '    + str(np.abs(et1_snrs_bilby)))
print('ET_ET1 GWBENCH: '    + str(np.abs(et1_snrs_gwbench)))
print(' ')
print('ET_ET2 Bilby: '    + str(np.abs(et2_snrs_bilby)))
print('ET_ET2 GWBENCH: '    + str(np.abs(et2_snrs_gwbench)))
print(' ')
print('ET_ET3 Bilby: '    + str(np.abs(et3_snrs_bilby)))
print('ET_ET3 GWBENCH: '    + str(np.abs(et3_snrs_gwbench)))


CE-40_C Bilby: [1271.42671315 1271.77429078 1274.86046859 1301.5449538  1297.91635226]
CE-40_C GWBENCH: [1271.54928959 1271.89690072 1274.98337607 1301.67043389 1298.04148252]
 
CE-20_S Bilby: [874.29946257 874.04784417 871.77335034 848.21399162 685.55214334]
CE-20_S GWBENCH: [874.38944248 874.13779819 871.86307027 848.3012869  685.62269802]
 
ET_ET1 Bilby: [443.61927245 443.63431157 443.76232671 444.23755322 339.4088244 ]
ET_ET1 GWBENCH: [443.69186597 443.70690755 443.83494364 444.31024791 339.464365  ]
 
ET_ET2 Bilby: [458.70539333 458.50868308 456.72734956 437.91488605 261.98486421]
ET_ET2 GWBENCH: [458.78045553 458.58371309 456.80208807 437.98654611 262.02773522]
 
ET_ET3 Bilby: [383.78308778 383.65658091 382.53262219 372.78250369 367.52092517]
ET_ET3 GWBENCH: [383.84588975 383.71936218 382.59521954 372.84350553 367.58106601]


## Bilby vs GWbench (Custom Detector Locations, LIGO)

### Bilby

In [6]:
minimum_frequency = 20
reference_frequency = 0.
duration = 256
sampling_frequency = 1024
approximant = "IMRPhenomD"

h1_snrs_bilby = []
l1_snrs_bilby = []
for time in times:
    injection_parameters = dict(
                                chirp_mass          = 30.9,
                                mass_ratio          = 0.8,
                                chi_1               = 0.0,
                                chi_2               = 0.0,
                                luminosity_distance = 400.0,
                                ra                  = np.pi/3,
                                dec                 = np.pi/3,
                                theta_jn            = np.pi/3,
                                psi                 = np.pi/3,
                                phase               = 0,
                                geocent_time        = time
    )

    # generate signal
    waveform_generator = bilby.gw.WaveformGenerator(
                                                    duration                      = duration,
                                                    sampling_frequency            = sampling_frequency,
                                                    frequency_domain_source_model = bilby.gw.source.lal_binary_black_hole,
                                                    waveform_arguments            = dict(waveform_approximant=approximant, reference_frequency=reference_frequency),
                                                    parameter_conversion          = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    )
    ifos = bilby.gw.detector.InterferometerList(["H1", "L1"])
    ifos.set_strain_data_from_power_spectral_densities(
                                                        sampling_frequency = sampling_frequency,
                                                        duration           = duration,
                                                        start_time         = injection_parameters["geocent_time"] - duration + 2,
    )
    ifos.inject_signal(waveform_generator=waveform_generator, parameters=injection_parameters)
    for ifo in ifos:
        ifo.minimum_frequency = minimum_frequency

    h1_snrs_bilby.append(ifos[0].meta_data['optimal_SNR'])
    l1_snrs_bilby.append(ifos[1].meta_data['optimal_SNR'])

h1_snrs_bilby = np.array(h1_snrs_bilby)
l1_snrs_bilby = np.array(l1_snrs_bilby)

13:34 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
13:34 bilby INFO    : Injected signal in H1:
13:34 bilby INFO    :   optimal SNR = 40.20
13:34 bilby INFO    :   matched filter SNR = 39.85+1.18j
13:34 bilby INFO    :   chirp_mass = 30.9
13:34 bilby INFO    :   mass_ratio = 0.8
13:34 bilby INFO    :   chi_1 = 0.0
13:34 bilby INFO    :   chi_2 = 0.0
13:34 bilby INFO    :   luminosity_distance = 400.0
13:34 bilby INFO    :   ra = 1.0471975511965976
13:34 bilby INFO    :   dec = 1.0471975511965976
13:34 bilby INFO    :   theta_jn = 1.0471975511965976
13:34 bilby INFO    :   psi = 1.0471975511965976
13:34 bilby INFO    :   phase = 0
13:34 bilby INFO    :   geocent_time = 0
13:34 bilby INFO    : Injected signal in L1:
13:34 bilby INFO    :   optimal SNR = 37.84
13:34 bilby INFO    :   matched fil

### GWbench

In [8]:
# Choose detector network
network_spec = ['aLIGO_H','aLIGO_L']

# Choose waveform.
wf_model_name = 'lal_bbh'
wf_other_var_dic = {'approximant':'IMRPhenomD'}

# Frequency range
freq_range = np.arange(20.,1024.,2**-4)

psd_dict  = {'psd_file':'aLIGO_O4_high_asd.txt','is_asd':1}
user_psds = {'aLIGO_H':psd_dict, 'aLIGO_L':psd_dict}

H1_loc = {'longitude':math.radians(-119.407842), 'latitide':math.radians(46.45513155), 'arm_azimuth': math.radians(-144.000758), 'which_arm': 'y', 'shape':'L'}
L1_loc = {'longitude':math.radians(-90.774240388), 'latitide':math.radians(30.562894333333), 'arm_azimuth': math.radians(-72.2836304), 'which_arm': 'y', 'shape':'L'}
user_locs = {'aLIGO_H': H1_loc, 'aLIGO_L': L1_loc}


net = network.Network(network_spec)
net.set_wf_vars(wf_model_name=wf_model_name, wf_other_var_dic=wf_other_var_dic)

h1_snrs_gwbench = []
l1_snrs_gwbench = []
for time in times:
    inj_params = {
        'Mc':    30.9,
        'eta':   0.247,
        'chi1x': 0.0,
        'chi1y': 0.0,
        'chi1z': 0.0,
        'chi2x': 0.0,
        'chi2y': 0.0,
        'chi2z': 0.0,
        'DL':    400,
        'tc':    time,
        'phic':  0.,
        'iota':  np.pi/3,
        'ra':    np.pi/3,
        'dec':   np.pi/3,
        'psi':   np.pi/3,
        'gmst0': bilby.gw.utils.greenwich_mean_sidereal_time(time)
        }

    net.set_net_vars(
        f=freq_range, inj_params=inj_params,
        deriv_symbs_string='Mc eta',
        use_rot=0, user_psds=user_psds, user_locs=user_locs
        )

    net.setup_ant_pat_lpf_psds() 
    net.calc_det_responses()
    net.calc_snrs()

    h1_snrs_gwbench.append(net.detectors[0].snr)
    l1_snrs_gwbench.append(net.detectors[1].snr)

h1_snrs_gwbench = np.array(h1_snrs_gwbench)
l1_snrs_gwbench = np.array(l1_snrs_gwbench)

2023-06-26 13:34:05,909 - Network - INFO : PSDs loaded.


2023-06-26 13:34:05,910 - Network - INFO : Antenna patterns and LPFs loaded.
2023-06-26 13:34:05,915 - Network - INFO : Detector responses calculated.
2023-06-26 13:34:05,917 - Network - INFO : SNRs calculated.
2023-06-26 13:34:05,933 - Network - INFO : PSDs loaded.
2023-06-26 13:34:05,940 - Network - INFO : Antenna patterns and LPFs loaded.
2023-06-26 13:34:05,951 - Network - INFO : Detector responses calculated.
2023-06-26 13:34:05,954 - Network - INFO : SNRs calculated.
2023-06-26 13:34:05,968 - Network - INFO : PSDs loaded.
2023-06-26 13:34:05,970 - Network - INFO : Antenna patterns and LPFs loaded.
2023-06-26 13:34:05,976 - Network - INFO : Detector responses calculated.
2023-06-26 13:34:05,978 - Network - INFO : SNRs calculated.
2023-06-26 13:34:05,993 - Network - INFO : PSDs loaded.
2023-06-26 13:34:05,995 - Network - INFO : Antenna patterns and LPFs loaded.
2023-06-26 13:34:06,000 - Network - INFO : Detector responses calculated.
2023-06-26 13:34:06,004 - Network - INFO : SNRs 

### Comparison

In [9]:
print('Hanford Bilby: '    + str(np.abs(h1_snrs_bilby)))
print('Hanford GWBENCH: '    + str(np.abs(h1_snrs_gwbench)))
print(' ')
print('Livingston Bilby: ' + str(np.abs(l1_snrs_bilby)))
print('Livingston GWBENCH: ' + str(np.abs(l1_snrs_gwbench)))

Hanford Bilby: [40.19676257 40.19698276 40.19969992 40.31364789 53.91190567 60.40783834]
Hanford GWBENCH: [40.19452118 40.19475908 40.1976365  40.31326548 53.92102874 60.4144077 ]
 
Livingston Bilby: [37.84044389 37.8502412  37.93925567 38.91493001 53.05739218 55.44002365]
Livingston GWBENCH: [37.8692365  37.87902306 37.96794198 38.94279342 53.08638296 55.4682317 ]
